#### Cryptography
#### Task № 5 - GSM   A5/1
#### Ivan Rybin - ITMO JB SE MA 2021

In [1]:
def xor_bits(l, r):
    return "".join([str(int(lb) ^ int(rb)) for lb, rb in zip(l, r)])

def stobytes(s):
    return s.encode('utf-8')

def bitsN(s, N):
    while len(s) % N != 0:
        s += '0'
    return s

def to8bits(bits):
    while len(bits) < 8:
        bits = '0' + bits
    return bits

def stobits(s):
    b = ""
    for c in stobytes(s):
        b += to8bits(bin(c)[2:])
    return b

def btoi(b):
    i = 0
    k = 0
    for c in reversed(b):
        i += int(c) * (2 ** k)
        k += 1
    return i

def bytestos(b):
    return b.decode('utf-8')

def split_to_blocks(b, block_size):
    blocks = []
    for i in range(1, int((len(b) + block_size) / block_size)):
        blocks.append(b[(i - 1) * block_size: i * block_size])
    return blocks

def bitstos(bits):
    s = []
    for b in split_to_blocks(bits, 8):
        s.append(btoi(b))
    return bytearray(s)

def cyclic_offset(l, offset, is_left=True):
    offset = offset % len(l)
    if is_left:
        return l[offset:] + l[:offset]
    return l[len(l)-offset:] + l[:len(l)-offset]

def xor_bits_list(l):
    return sum(l) % 2

In [2]:
tapped_bits = [
    [13, 16, 17, 18],
    [20, 21],
    [7, 20, 21, 22]
]

clock_bits = [8, 10, 11]


def drop_registers():
    R1, R2, R3 = [0] * 19, [0] * 22, [0] * 23
    LFSRs = [R1, R2, R3]
    return LFSRs


def F_func(LFSRs):
    x = LFSRs[0][-(clock_bits[0] + 1)]
    y = LFSRs[1][-(clock_bits[1] + 1)]
    z = LFSRs[2][-(clock_bits[2] + 1)]
    return (x & y) | (x & z) | (y & z)


def one_clock(LFSRs):
    sync_bits = [0, 0, 0]
    for i in range(0, 3):
        sync_bits[i] = xor_bits_list(
            [LFSRs[i][-(tid + 1)] for tid in tapped_bits[i]]
        )
    return sync_bits


def init_registers(LFSRs, key64_bits, frame):
    assert len(key64_bits) == 64 and len(frame) == 22
    
    for b in key64_bits + frame:
        sync_bits = one_clock(LFSRs)
        for i in range(0, 3):
            LFSRs[i] = LFSRs[i][1:] + [sync_bits[i] ^ b]
    return LFSRs

def irregular_clocking(LFSRs):
    sync_bits = one_clock(LFSRs)
    F_val = F_func(LFSRs)
    for i in range(0, 3):
        if sync_bits[i] == F_val:
            LFSRs[i] = LFSRs[i][1:] + [sync_bits[i]]
    return LFSRs


def print_LFSRs(LFSRs):
    for i in range(0, 3):
        print(''.join([str(b) for b in LFSRs[i]]))
    print('-' * 10)
            
            
def GSM_A5_1(message, key64):
    key64 = [int(b) for b in key64]
    
    processed = ''
    for i in range(0, int(len(message) / 114)):
        FRAME_N = [int(b) for b in bitsN(bin(i + 1)[2:], 22)]
        LFSRs = drop_registers()
        
        # 64 + 22 clocks
        LFSRs = init_registers(LFSRs, key64, FRAME_N)
        
        # 100 clocks
        for j in range(0, 100):
            LFSRs = irregular_clocking(LFSRs)
            
        # 114 bits key stream
        key_stream = []
        for j in range(0, 114):
            LFSRs = irregular_clocking(LFSRs)
            encrypt_bit = LFSRs[0][0] ^ LFSRs[1][0] ^ LFSRs[2][0]
            key_stream.append(encrypt_bit)
            

        src_block = message[i*114:(i + 1)*114]
        key_block = ''.join([str(b) for b in key_stream])
        
        processed += xor_bits(src_block, key_block)

    return processed

In [3]:
def run_all():
    data = 'this is my message 123 HELLO'
    GSM_key = 'crypto42'
    
    data_bits = stobits(data)
    data_bits_114 = bitsN(data_bits, 114)
    
    GSM_key_bits = stobits(GSM_key)
    
    encrypted = GSM_A5_1(data_bits_114, GSM_key_bits)
    decrypted = GSM_A5_1(encrypted, GSM_key_bits)

    
    print(f'msg: {data}')
    print(f'msg len: {len(data)}')
    print(f'msg bits len: {len(data_bits_114)}')
    print(f'msg bits: {data_bits_114}\n')
    
    print(f'enc key: {GSM_key}')
    print(f'key len: {len(GSM_key)}')
    print(f'len bits: {len(GSM_key_bits)}')
    print(f'key bits: {GSM_key_bits}\n')
        
    print('ENCRYPTED')
    print(f'enc len: {len(encrypted)}')
    print(f'enc: {encrypted}\n')
    
    print('DECRYPTED')
    print(f'IS ENCRYPTED != MSG BITS: {data_bits_114 != encrypted}')
    print(f'IS DECRYPTED == MSG BITS: {data_bits_114 == decrypted}\n')
    print(f'dec len: {len(decrypted)}')
    print(f'dec bits: {decrypted}\n')
    print(f'dec msg: {bytestos(bitstos(decrypted[:len(data) * 8]))}')

In [4]:
run_all()

msg: this is my message 123 HELLO
msg len: 28
msg bits len: 228
msg bits: 011101000110100001101001011100110010000001101001011100110010000001101101011110010010000001101101011001010111001101110011011000010110011101100101001000000011000100110010001100110010000001001000010001010100110001001100010011110000

enc key: crypto42
key len: 8
len bits: 64
key bits: 0110001101110010011110010111000001110100011011110011010000110010

ENCRYPTED
enc len: 228
enc: 100010111001011110010110100011001101111110010110100011001101111110010010100001101101111110010010100110101000110010001100100111101001100010011010110111111100111011001101110011001101111110110111101110101011001110110011101100001111

DECRYPTED
IS ENCRYPTED != MSG BITS: True
IS DECRYPTED == MSG BITS: True

dec len: 228
dec bits: 01110100011010000110100101110011001000000110100101110011001000000110110101111001001000000110110101100101011100110111001101100001011001110110010100100000001100010011001000110011001000000100100001000101010011000100110001001111